# Install Packages and Libraries

In [1]:
!pip install numpy rasterio scipy scikit-learn matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 69.3 MB/s eta 0:00:00


In [2]:
import numpy as np
import rasterio
from scipy.ndimage import convolve
from scipy.ndimage import gaussian_filter
from scipy.ndimage import uniform_filter
from sklearn.metrics import roc_curve, precision_recall_curve
import matplotlib.pyplot as plt
from scipy.special import expit

# Methods

In [3]:
# -----------------------------
# Methods/tfxn.py
# -----------------------------

# Define the function f
def f(a):
    """
    Compute the function f(a) = -ln(1 + exp(a))

    Parameters:
    a : np.ndarray
        Input array.

    Returns:
    np.ndarray
        Output after applying the function.
    """

    return -np.log(1 + np.exp(a))

# Define the derivative of function f
def df(a):
    """
    Compute the derivative of f(a), which is df(a)/da = 1 / (1 + exp(a))

    Parameters:
    a : np.ndarray
        Input array.

    Returns:
    np.ndarray
        Derivative of f with respect to a.
    """

    return 1 / (1 + np.exp(a))

# Nonlinear Function T
def Tfxn(y, qBD, qLS, qLF, alLS, alLF, w, local, delta):
    """
    Nonlinear Transformation Function T

    Parameters:
    y     : np.ndarray
        Observed data.
    qBD   : np.ndarray
        Posterior probability for BD.
    qLS   : np.ndarray
        Posterior probability for LS.
    qLF   : np.ndarray
        Posterior probability for LF.
    alLS  : np.ndarray
        Alpha parameter for LS.
    alLF  : np.ndarray
        Alpha parameter for LF.
    w     : np.ndarray
        Weight vector (array of length 15).
    local : np.ndarray
        Local model classification.
    delta : float
        Tolerance parameter.

    Returns:
    np.ndarray
        Combined gradients for BD, LS, and LF.
    """

    # Unpack weights
    w0, weps, w0BD, w0LS, w0LF, wLSBD, wLFBD, wBDy, wLSy, wLFy, weLS, weLF, weBD, waLS, waLF = w

    y = np.log(1e-6 + y)

    # Initialize gBD, gLS, gLF
    gBD = np.zeros_like(y)
    gLS = np.zeros_like(y)
    gLF = np.zeros_like(y)

    # Compute gBD
    idx_local3 = (local == 3)
    idx_local4 = (local == 4)
    idx_local6 = (local == 6)
    idx_local3_4_6 = (local == 3) | (local == 4) | (local == 6)

    weBD_sq_div2 = (w[12] ** 2) / 2  # w[12] is weBD

    # Local == 3
    gBD[idx_local3] += (
        qLS[idx_local3] * f(- w[2] - w[5] + weBD_sq_div2)
        + (1 - qLS[idx_local3]) * f(- w[2] + weBD_sq_div2)
        - qLS[idx_local3] * f(w[2] + w[5] + weBD_sq_div2)
        - (1 - qLS[idx_local3]) * f(w[2] + weBD_sq_div2)
    )

    # Local == 4
    gBD[idx_local4] += (
        qLF[idx_local4] * f(- w[2] - w[6] + weBD_sq_div2)
        + (1 - qLF[idx_local4]) * f(- w[2] + weBD_sq_div2)
        - qLF[idx_local4] * f(w[2] + w[6] + weBD_sq_div2)
        - (1 - qLF[idx_local4]) * f(w[2] + weBD_sq_div2)
    )

    # Subtract terms involving y, wBDy, etc.
    term = ((w[7] - 2 * y + 2 * w[0]) / (2 * (w[1] ** 2))) * w[7]
    gBD[idx_local3_4_6] -= term[idx_local3_4_6]

    # Additional terms for gBD
    if w[1] ** 2 != 0:
        gBD[idx_local3] -= (1 / (w[1] ** 2)) * (w[8] * w[7] * qLS[idx_local3])
        gBD[idx_local4] -= (1 / (w[1] ** 2)) * (w[9] * w[7] * qLF[idx_local4])
        gBD[idx_local6] -= (1 / (w[1] ** 2)) * (w[8] * w[9] * w[7] * qLS[idx_local6] * qLF[idx_local6])

    # Additional terms for local == 6
    if any(idx_local6):
        a1 = - w[2] - w[5] - w[6] + weBD_sq_div2
        a2 = - w[2] - w[5] + weBD_sq_div2
        a3 = - w[2] - w[6] + weBD_sq_div2
        a4 = w[2] + weBD_sq_div2
        a5 = w[2] + w[5] + w[6] + weBD_sq_div2
        a6 = w[2] + w[5] + weBD_sq_div2
        a7 = w[2] + w[6] + weBD_sq_div2

        qLS_local6 = qLS[idx_local6]
        qLF_local6 = qLF[idx_local6]

        term9 = (
            (qLS_local6 * qLF_local6 * f(a1)
             + qLS_local6 * (1 - qLF_local6) * f(a2)
             + (1 - qLS_local6) * qLF_local6 * f(a3)
             - (1 - qLS_local6) * (1 - qLF_local6) * f(a4))
            - (qLS_local6 * qLF_local6 * f(a5)
               - qLS_local6 * (1 - qLF_local6) * f(a6)
               - (1 - qLS_local6) * qLF_local6 * f(a7)
               - (1 - qLS_local6) * (1 - qLF_local6) * f(a4))
        )
        gBD[idx_local6] -= term9

    # Compute gLS
    idx_local1_3_5_6 = (local == 1) | (local == 3) | (local == 5) | (local == 6)
    weLS_sq_div2 = (w[10] ** 2) / 2

    gLS[idx_local1_3_5_6] += (
        f(- w[3] - w[13] * alLS[idx_local1_3_5_6] + weLS_sq_div2)
        - f(w[3] + w[13] * alLS[idx_local1_3_5_6] + weLS_sq_div2)
    )

    idx_local3 = (local == 3)
    idx_local5 = (local == 5)
    idx_local6 = (local == 6)

    # Additional terms for gLS
    if any(idx_local3):
        gLS[idx_local3] += (
            qBD[idx_local3] * (f(- w[2] - w[5] + weBD_sq_div2) - f(- w[2] + weBD_sq_div2))
            + (1 - qBD[idx_local3]) * (f(w[2] + w[5] + weBD_sq_div2) - f(w[2] + weBD_sq_div2))
        )

    if any(idx_local6):
        qBD_local6 = qBD[idx_local6]
        qLF_local6 = qLF[idx_local6]

        term = (
            qBD_local6 * (
                qLF_local6 * f(- w[2] - w[5] - w[6] + weBD_sq_div2)
                + (1 - qLF_local6) * f(- w[2] - w[5] + weBD_sq_div2)
                - qLF_local6 * f(- w[2] - w[6] + weBD_sq_div2)
                - (1 - qLF_local6) * f(- w[2] + weBD_sq_div2)
            )
            + (1 - qBD_local6) * (
                qLF_local6 * f(w[2] + w[5] + w[6] + weBD_sq_div2)
                + (1 - qLF_local6) * f(w[2] + w[5] + weBD_sq_div2)
                - qLF_local6 * f(w[2] + w[6] + weBD_sq_div2)
                - (1 - qLF_local6) * f(w[2] + weBD_sq_div2)
            )
        )
        gLS[idx_local6] += term

    # Subtract terms involving y, wLSy, etc.
    term = ((w[8] - 2 * y + 2 * w[0]) / (2 * (w[1] ** 2))) * w[8]
    gLS[idx_local1_3_5_6] -= term[idx_local1_3_5_6]

    if w[1] ** 2 != 0:
        gLS[idx_local3] -= (1 / (w[1] ** 2)) * (w[7] * w[8] * qBD[idx_local3])
        gLS[idx_local5] -= (1 / (w[1] ** 2)) * (w[8] * w[9] * qLF[idx_local5])
        gLS[idx_local6] -= (1 / (w[1] ** 2)) * (w[7] * w[8] * w[9] * qBD[idx_local6] * qLF[idx_local6])

    # Additional terms for gLS
    idx_local5_or_6 = (local == 5) | (local == 6)
    gLS[idx_local5_or_6] -= qLF[idx_local5_or_6] / (2 * delta)

    # Compute gLF similarly
    idx_local2_4_5_6 = (local == 2) | (local == 4) | (local == 5) | (local == 6)
    weLF_sq_div2 = (w[11] ** 2) / 2

    gLF[idx_local2_4_5_6] += (
        f(- w[4] - w[14] * alLF[idx_local2_4_5_6] + weLF_sq_div2)
        - f(w[4] + w[14] * alLF[idx_local2_4_5_6] + weLF_sq_div2)
    )

    idx_local4 = (local == 4)
    idx_local5 = (local == 5)
    idx_local6 = (local == 6)

    if any(idx_local4):
        gLF[idx_local4] += (
            qBD[idx_local4] * (f(- w[2] - w[6] + weBD_sq_div2) - f(- w[2] + weBD_sq_div2))
            + (1 - qBD[idx_local4]) * (f(w[2] + w[6] + weBD_sq_div2) - f(w[2] + weBD_sq_div2))
        )

    if any(idx_local6):
        qBD_local6 = qBD[idx_local6]
        qLS_local6 = qLS[idx_local6]

        term = (
            qBD_local6 * (
                qLS_local6 * f(- w[2] - w[5] - w[6] + weBD_sq_div2)
                + (1 - qLS_local6) * f(- w[2] - w[6] + weBD_sq_div2)
                - qLS_local6 * f(- w[2] - w[5] + weBD_sq_div2)
                - (1 - qLS_local6) * f(- w[2] + weBD_sq_div2)
            )
            + (1 - qBD_local6) * (
                qLS_local6 * f(w[2] + w[5] + w[6] + weBD_sq_div2)
                + (1 - qLS_local6) * f(w[2] + w[6] + weBD_sq_div2)
                - qLS_local6 * f(w[2] + w[5] + weBD_sq_div2)
                - (1 - qLS_local6) * f(w[2] + weBD_sq_div2)
            )
        )
        gLF[idx_local6] += term

    # Subtract terms involving y, wLFy, etc.
    term = ((w[9] - 2 * y + 2 * w[0]) / (2 * (w[1] ** 2))) * w[9]
    gLF[idx_local2_4_5_6] -= term[idx_local2_4_5_6]

    if w[1] ** 2 != 0:
        gLF[idx_local4] -= (1 / (w[1] ** 2)) * (w[7] * w[9] * qBD[idx_local4])
        gLF[idx_local5] -= (1 / (w[1] ** 2)) * (w[8] * w[9] * qLS[idx_local5])
        gLF[idx_local6] -= (1 / (w[1] ** 2)) * (w[7] * w[8] * w[9] * qBD[idx_local6] * qLS[idx_local6])

    # Additional terms for gLF
    idx_local5_or_6 = (local == 5) | (local == 6)
    gLF[idx_local5_or_6] -= qLS[idx_local5_or_6] / (2 * delta)

    # Combine
    g = np.column_stack((gBD, gLS, gLF))

    return g

In [4]:
# -----------------------------
# Methods/loss.py
# -----------------------------

# Define the Loss function
def Loss(y, qBD, qLS, qLF, alLS, alLF, w, local, delta):
    """
    Compute the Loss function

    Parameters:
    y     : np.ndarray
        Observed data.
    qBD   : np.ndarray
        Posterior probability for BD.
    qLS   : np.ndarray
        Posterior probability for LS.
    qLF   : np.ndarray
        Posterior probability for LF.
    alLS  : np.ndarray
        Alpha parameter for LS.
    alLF  : np.ndarray
        Alpha parameter for LF.
    w     : np.ndarray
        Weight vector (array of length 15).
    local : np.ndarray
        Local model classification.
    delta : float
        Tolerance parameter.

    Returns:
    np.ndarray
        Total loss.
    """
    # Unpack weights
    w0, weps, w0BD, w0LS, w0LF, wLSBD, wLFBD, wBDy, wLSy, wLFy, weLS, weLF, weBD, waLS, waLF = w

    y = np.log(1e-6 + y)

    # Clip qBD, qLS, qLF to avoid log(0)
    f_qBD = np.clip(qBD, 1e-6, 1 - 1e-6)
    f_qLS = np.clip(qLS, 1e-6, 1 - 1e-6)
    f_qLF = np.clip(qLF, 1e-6, 1 - 1e-6)

    # Initialize LBD, LLS, LLF
    LBD = np.zeros_like(y)
    LLS = np.zeros_like(y)
    LLF = np.zeros_like(y)

    idx_local3 = (local == 3)
    idx_local4 = (local == 4)
    idx_local6 = (local == 6)

    weBD_sq_div2 = (weBD ** 2) / 2

    # Compute LBD
    if any(idx_local3):
        LBD[idx_local3] += (
            qBD[idx_local3] * (qLS[idx_local3] * f(- w0BD - wLSBD + weBD_sq_div2) + (1 - qLS[idx_local3]) * f(-w0BD + weBD_sq_div2))
            + (1 - qBD[idx_local3]) * (qLS[idx_local3] * f(w0BD + wLSBD + weBD_sq_div2) + (1 - qLS[idx_local3]) * f(w0BD + weBD_sq_div2))
            - (qBD[idx_local3] * np.log(f_qBD[idx_local3]) + (1 - qBD[idx_local3]) * np.log(1 - f_qBD[idx_local3]))
        )

    if any(idx_local4):
        LBD[idx_local4] += (
            qBD[idx_local4] * (qLF[idx_local4] * f(- w0BD - wLFBD + weBD_sq_div2) + (1 - qLF[idx_local4]) * f(-w0BD + weBD_sq_div2))
            + (1 - qBD[idx_local4]) * (qLF[idx_local4] * f(w0BD + wLFBD + weBD_sq_div2) + (1 - qLF[idx_local4]) * f(w0BD + weBD_sq_div2))
            - (qBD[idx_local4] * np.log(f_qBD[idx_local4]) + (1 - qBD[idx_local4]) * np.log(1 - f_qBD[idx_local4]))
        )

    if any(idx_local6):
        qLS_local6 = qLS[idx_local6]
        qLF_local6 = qLF[idx_local6]
        qBD_local6 = qBD[idx_local6]
        term1 = (
            qBD_local6 * (
                qLS_local6 * qLF_local6 * f(- w0BD - wLSBD - wLFBD + weBD_sq_div2)
                + qLS_local6 * (1 - qLF_local6) * f(- w0BD - wLSBD + weBD_sq_div2)
                + (1 - qLS_local6) * qLF_local6 * f(- w0BD - wLFBD + weBD_sq_div2)
                + (1 - qLS_local6) * (1 - qLF_local6) * f(- w0BD + weBD_sq_div2)
            )
            + (1 - qBD_local6) * (
                qLS_local6 * qLF_local6 * f(w0BD + wLSBD + wLFBD + weBD_sq_div2)
                + qLS_local6 * (1 - qLF_local6) * f(w0BD + wLSBD + weBD_sq_div2)
                + (1 - qLS_local6) * qLF_local6 * f(w0BD + wLFBD + weBD_sq_div2)
                + (1 - qLS_local6) * (1 - qLF_local6) * f(w0BD + weBD_sq_div2)
            )
            - (qBD_local6 * np.log(f_qBD[idx_local6]) + (1 - qBD_local6) * np.log(1 - f_qBD[idx_local6]))
        )
        LBD[idx_local6] += term1

    # Compute LLS
    idx_local1_3_5_6 = (local == 1) | (local == 3) | (local == 5) | (local == 6)
    if any(idx_local1_3_5_6):
        LLS[idx_local1_3_5_6] += (
            qLS[idx_local1_3_5_6] * f(- w0LS - waLS * alLS[idx_local1_3_5_6] + (weLS ** 2) / 2)
            + (1 - qLS[idx_local1_3_5_6]) * f(w0LS + waLS * alLS[idx_local1_3_5_6] + (weLS ** 2) / 2)
            - (qLS[idx_local1_3_5_6] * np.log(f_qLS[idx_local1_3_5_6]) + (1 - qLS[idx_local1_3_5_6]) * np.log(1 - f_qLS[idx_local1_3_5_6]))
        )

    # Compute LLF similarly
    idx_local2_4_5_6 = (local == 2) | (local == 4) | (local == 5) | (local == 6)
    if any(idx_local2_4_5_6):
        LLF[idx_local2_4_5_6] += (
            qLF[idx_local2_4_5_6] * f(- w0LF - waLF * alLF[idx_local2_4_5_6] + (weLF ** 2) / 2)
            + (1 - qLF[idx_local2_4_5_6]) * f(w0LF + waLF * alLF[idx_local2_4_5_6] + (weLF ** 2) / 2)
            - (qLF[idx_local2_4_5_6] * np.log(f_qLF[idx_local2_4_5_6]) + (1 - qLF[idx_local2_4_5_6]) * np.log(1 - f_qLF[idx_local2_4_5_6]))
        )

    # Compute Leps
    Leps = - (0.5 * np.log(weps ** 2) + (y - w0) ** 2 / (2 * (weps ** 2)))
    term = (1 / (2 * (weps ** 2))) * (
        ((local == 3) | (local == 4) | (local == 6)) * wBDy * qBD * (wBDy - 2 * y + 2 * w0)
        + ((local == 1) | (local == 3) | (local == 5) | (local == 6)) * wLSy * qLS * (wLSy - 2 * y + 2 * w0)
        + ((local == 2) | (local == 4) | (local == 5) | (local == 6)) * wLFy * qLF * (wLFy - 2 * y + 2 * w0)
    )
    Leps -= term

    Leps -= (1 / (weps ** 2)) * (
        ((local == 3) * wBDy * wLSy * qBD * qLS)
        + ((local == 4) * wBDy * wLFy * qBD * qLF)
        + ((local == 5) * wLSy * wLFy * qLS * qLF)
        + ((local == 6) * wBDy * wLSy * wLFy * qBD * qLS * qLF)
    )

    Leps -= y

    # L_ex
    idx_local5_6 = (local == 5) | (local == 6)
    L_ex = np.zeros_like(y)
    L_ex[idx_local5_6] = - (qLS[idx_local5_6] * qLF[idx_local5_6]) / (2 * delta)
    L_ex = np.maximum(L_ex, -1e+2)

    # Total Loss
    L = LBD + LLS + LLF + Leps + L_ex

    return L


In [5]:
# -----------------------------
# Methods/parder.py
# -----------------------------

def parder(y, qBD, qLS, qLF, alLS, alLF, w, local):
    """
    Compute the partial derivatives (gradients) for the optimization process.

    Parameters:
    y     : Observed data (array)
    qBD   : Posterior probability for BD (array)
    qLS   : Posterior probability for LS (array)
    qLF   : Posterior probability for LF (array)
    alLS  : Alpha parameter for LS (array)
    alLF  : Alpha parameter for LF (array)
    w     : Weight vector (array of length 15)
    local : Local model classification (array)

    Returns:
    grad  : Gradient matrix (array of shape (len(y), 15))
    """

    # Unpack weights
    w0, weps, w0BD, w0LS, w0LF, wLSBD, wLFBD, wBDy, wLSy, wLFy, weLS, weLF, weBD, waLS, waLF = w

    y = np.log(1e-6 + y)

    # Initialize gradient matrix
    grad = np.zeros((len(y), 15))

    # Precompute common terms
    weps_sq = weps ** 2
    weBD_sq_div2 = (weBD ** 2) / 2
    weLS_sq_div2 = (weLS ** 2) / 2
    weLF_sq_div2 = (weLF ** 2) / 2

    # Compute indicators
    idx_local1 = (local == 1)
    idx_local2 = (local == 2)
    idx_local3 = (local == 3)
    idx_local4 = (local == 4)
    idx_local5 = (local == 5)
    idx_local6 = (local == 6)

    # grad[:, 0] corresponds to w0y
    grad[:, 0] = (1 / weps_sq) * (y - w0)
    grad[:, 0] -= ((idx_local3 | idx_local4 | idx_local6) * (1 / weps_sq) * (wBDy * qBD))
    grad[:, 0] -= ((idx_local1 | idx_local3 | idx_local5 | idx_local6) * (1 / weps_sq) * (wLSy * qLS))
    grad[:, 0] -= ((idx_local2 | idx_local4 | idx_local5 | idx_local6) * (1 / weps_sq) * (wLFy * qLF))

    # grad[:, 1] corresponds to wey
    term = -1 / weps - (1 / weps ** 3) * (
        - y ** 2 - w0 ** 2 + 2 * w0 * y
        - ((idx_local1 | idx_local3 | idx_local5 | idx_local6) * (wLSy ** 2 - 2 * y * wLSy + 2 * w0 * wLSy) * qLS)
        - ((idx_local2 | idx_local4 | idx_local5 | idx_local6) * (wLFy ** 2 - 2 * y * wLFy + 2 * w0 * wLFy) * qLF)
        - ((idx_local3 | idx_local4 | idx_local6) * (wBDy ** 2 - 2 * y * wBDy + 2 * w0 * wBDy) * qBD)
        - (idx_local3 * 2 * (wBDy * wLSy * qLS * qBD))
        - (idx_local4 * 2 * (wBDy * wLFy * qLF * qBD))
        - (idx_local6 * 2 * (wBDy * wLSy * wLFy * qLS * qLF * qBD))
    )
    grad[:, 1] = term

    # grad[:, 2] corresponds to w0BD
    grad[:, 2] = 0
    if np.any(idx_local3):
        grad[idx_local3, 2] += (
            qBD[idx_local3] * qLS[idx_local3] * df(w0BD + wLSBD - weBD_sq_div2)
            - (1 - qBD[idx_local3]) * qLS[idx_local3] * df(-w0BD - wLSBD - weBD_sq_div2)
            + qBD[idx_local3] * (1 - qLS[idx_local3]) * df(w0BD - weBD_sq_div2)
            - (1 - qBD[idx_local3]) * (1 - qLS[idx_local3]) * df(-w0BD - weBD_sq_div2)
        )

    if np.any(idx_local4):
        grad[idx_local4, 2] += (
            qBD[idx_local4] * qLF[idx_local4] * df(w0BD + wLFBD - weBD_sq_div2)
            - (1 - qBD[idx_local4]) * qLF[idx_local4] * df(-w0BD - wLFBD - weBD_sq_div2)
            + qBD[idx_local4] * (1 - qLF[idx_local4]) * df(w0BD - weBD_sq_div2)
            - (1 - qBD[idx_local4]) * (1 - qLF[idx_local4]) * df(-w0BD - weBD_sq_div2)
        )

    if np.any(idx_local6):
        qBD_local6 = qBD[idx_local6]
        qLS_local6 = qLS[idx_local6]
        qLF_local6 = qLF[idx_local6]

        term = (
            qBD_local6 * qLS_local6 * qLF_local6 * df(w0BD + wLSBD + wLFBD - weBD_sq_div2)
            + qBD_local6 * qLS_local6 * (1 - qLF_local6) * df(w0BD + wLSBD - weBD_sq_div2)
            + qBD_local6 * (1 - qLS_local6) * qLF_local6 * df(w0BD + wLFBD - weBD_sq_div2)
            + qBD_local6 * (1 - qLS_local6) * (1 - qLF_local6) * df(w0BD - weBD_sq_div2)
            - (1 - qBD_local6) * qLS_local6 * qLF_local6 * df(-w0BD - wLSBD - wLFBD - weBD_sq_div2)
            - (1 - qBD_local6) * qLS_local6 * (1 - qLF_local6) * df(-w0BD - wLSBD - weBD_sq_div2)
            - (1 - qBD_local6) * (1 - qLS_local6) * qLF_local6 * df(-w0BD - wLFBD - weBD_sq_div2)
            - (1 - qBD_local6) * (1 - qLS_local6) * (1 - qLF_local6) * df(-w0BD - weBD_sq_div2)
        )
        grad[idx_local6, 2] += term

    # grad[:, 3] corresponds to w0LS
    idx_LS = idx_local1 | idx_local3 | idx_local5 | idx_local6
    grad[:, 3] = 0
    grad[idx_LS, 3] += (
        qLS[idx_LS] * df(w0LS + waLS * alLS[idx_LS] - weLS_sq_div2)
        - (1 - qLS[idx_LS]) * df(-w0LS - waLS * alLS[idx_LS] - weLS_sq_div2)
    )

    # grad[:, 4] corresponds to w0LF
    idx_LF = idx_local2 | idx_local4 | idx_local5 | idx_local6
    grad[:, 4] = 0
    grad[idx_LF, 4] += (
        qLF[idx_LF] * df(w0LF + waLF * alLF[idx_LF] - weLF_sq_div2)
        - (1 - qLF[idx_LF]) * df(-w0LF - waLF * alLF[idx_LF] - weLF_sq_div2)
    )

    # grad[:, 5] corresponds to wLSBD
    grad[:, 5] = 0
    if np.any(idx_local3):
        grad[idx_local3, 5] += (
            qBD[idx_local3] * qLS[idx_local3] * df(w0BD + wLSBD - weBD_sq_div2)
            - (1 - qBD[idx_local3]) * qLS[idx_local3] * df(-w0BD - wLSBD - weBD_sq_div2)
        )

    if np.any(idx_local6):
        grad[idx_local6, 5] += (
            qBD[idx_local6] * qLS[idx_local6] * qLF[idx_local6] * df(w0BD + wLSBD + wLFBD - weBD_sq_div2)
            + qBD[idx_local6] * qLS[idx_local6] * (1 - qLF[idx_local6]) * df(w0BD + wLSBD - weBD_sq_div2)
            - (1 - qBD[idx_local6]) * qLS[idx_local6] * qLF[idx_local6] * df(-w0BD - wLSBD - wLFBD - weBD_sq_div2)
            - (1 - qBD[idx_local6]) * qLS[idx_local6] * (1 - qLF[idx_local6]) * df(-w0BD - wLSBD - weBD_sq_div2)
        )

    # grad[:, 6] corresponds to wLFBD
    grad[:, 6] = 0
    if np.any(idx_local4):
        grad[idx_local4, 6] += (
            qBD[idx_local4] * qLF[idx_local4] * df(w0BD + wLFBD - weBD_sq_div2)
            - (1 - qBD[idx_local4]) * qLF[idx_local4] * df(-w0BD - wLFBD - weBD_sq_div2)
        )

    if np.any(idx_local6):
        grad[idx_local6, 6] += (
            qBD[idx_local6] * qLF[idx_local6] * qLS[idx_local6] * df(w0BD + wLSBD + wLFBD - weBD_sq_div2)
            + qBD[idx_local6] * qLF[idx_local6] * (1 - qLS[idx_local6]) * df(w0BD + wLFBD - weBD_sq_div2)
            - (1 - qBD[idx_local6]) * qLF[idx_local6] * qLS[idx_local6] * df(-w0BD - wLSBD - wLFBD - weBD_sq_div2)
            - (1 - qBD[idx_local6]) * qLF[idx_local6] * (1 - qLS[idx_local6]) * df(-w0BD - wLFBD - weBD_sq_div2)
        )

    # grad[:, 7] corresponds to wBDy
    idx_BDy = idx_local3 | idx_local4 | idx_local6
    grad[:, 7] = 0
    grad[idx_BDy, 7] += (-1 / weps_sq) * qBD[idx_BDy] * (wBDy - y[idx_BDy] + w0)
    grad[idx_BDy, 7] -= (1 / weps_sq) * (
        (idx_local3 * wLSy * qBD * qLS)
        + (idx_local4 * wLFy * qBD * qLF)
        + (idx_local6 * wLSy * wLFy * qBD * qLS * qLF)
    )[idx_BDy]

    # grad[:, 8] corresponds to wLSy
    idx_LSy = idx_local1 | idx_local3 | idx_local5 | idx_local6
    grad[:, 8] = 0
    grad[idx_LSy, 8] += (-1 / weps_sq) * qLS[idx_LSy] * (wLSy - y[idx_LSy] + w0)
    grad[idx_LSy, 8] -= (1 / weps_sq) * (
        (idx_local3 * wBDy * qBD * qLS)
        + (idx_local6 * wBDy * wLFy * qBD * qLS * qLF)
    )[idx_LSy]

    # grad[:, 9] corresponds to wLFy
    idx_LFy = idx_local2 | idx_local4 | idx_local5 | idx_local6
    grad[:, 9] = 0
    grad[idx_LFy, 9] += (-1 / weps_sq) * qLF[idx_LFy] * (wLFy - y[idx_LFy] + w0)
    grad[idx_LFy, 9] -= (1 / weps_sq) * (
        (idx_local4 * wBDy * qBD * qLF)
        + (idx_local6 * wBDy * wLSy * qBD * qLF * qLS)
    )[idx_LFy]

    # grad[:, 10] corresponds to weLS
    grad[:, 10] = 0
    grad[idx_LS, 10] += (
        - qLS[idx_LS] * weLS * df(w0LS + waLS * alLS[idx_LS] - weLS_sq_div2)
        - (1 - qLS[idx_LS]) * weLS * df(-w0LS - waLS * alLS[idx_LS] - weLS_sq_div2)
    )

    # grad[:, 11] corresponds to weLF
    grad[:, 11] = 0
    grad[idx_LF, 11] += (
        - qLF[idx_LF] * weLF * df(w0LF + waLF * alLF[idx_LF] - weLF_sq_div2)
        - (1 - qLF[idx_LF]) * weLF * df(-w0LF - waLF * alLF[idx_LF] - weLF_sq_div2)
    )

    # grad[:, 12] corresponds to weBD
    grad[:, 12] = 0
    if np.any(idx_local3):
        grad[idx_local3, 12] += (
            - qBD[idx_local3] * qLS[idx_local3] * weBD * df(w0BD + wLSBD - weBD_sq_div2)
            - (1 - qBD[idx_local3]) * qLS[idx_local3] * weBD * df(-w0BD - wLSBD - weBD_sq_div2)
            - qBD[idx_local3] * (1 - qLS[idx_local3]) * weBD * df(w0BD - weBD_sq_div2)
            - (1 - qBD[idx_local3]) * (1 - qLS[idx_local3]) * weBD * df(-w0BD - weBD_sq_div2)
        )

    if np.any(idx_local4):
        grad[idx_local4, 12] += (
            - qBD[idx_local4] * qLF[idx_local4] * weBD * df(w0BD + wLFBD - weBD_sq_div2)
            - (1 - qBD[idx_local4]) * qLF[idx_local4] * weBD * df(-w0BD - wLFBD - weBD_sq_div2)
            - qBD[idx_local4] * (1 - qLF[idx_local4]) * weBD * df(w0BD - weBD_sq_div2)
            - (1 - qBD[idx_local4]) * (1 - qLF[idx_local4]) * weBD * df(-w0BD - weBD_sq_div2)
        )

    if np.any(idx_local6):
        grad[idx_local6, 12] += (
            - qBD[idx_local6] * qLS[idx_local6] * qLF[idx_local6] * weBD * df(w0BD + wLSBD + wLFBD - weBD_sq_div2)
            - qBD[idx_local6] * qLS[idx_local6] * (1 - qLF[idx_local6]) * weBD * df(w0BD + wLSBD - weBD_sq_div2)
            - qBD[idx_local6] * (1 - qLS[idx_local6]) * qLF[idx_local6] * weBD * df(w0BD + wLFBD - weBD_sq_div2)
            - qBD[idx_local6] * (1 - qLS[idx_local6]) * (1 - qLF[idx_local6]) * weBD * df(w0BD - weBD_sq_div2)
            - (1 - qBD[idx_local6]) * qLS[idx_local6] * qLF[idx_local6] * weBD * df(-w0BD - wLSBD - wLFBD - weBD_sq_div2)
            - (1 - qBD[idx_local6]) * qLS[idx_local6] * (1 - qLF[idx_local6]) * weBD * df(-w0BD - wLSBD - weBD_sq_div2)
            - (1 - qBD[idx_local6]) * (1 - qLS[idx_local6]) * qLF[idx_local6] * weBD * df(-w0BD - wLFBD - weBD_sq_div2)
            - (1 - qBD[idx_local6]) * (1 - qLS[idx_local6]) * (1 - qLF[idx_local6]) * weBD * df(-w0BD - weBD_sq_div2)
        )

    # grad[:, 13] corresponds to waLS
    grad[:, 13] = 0
    grad[idx_LS, 13] += (
        qLS[idx_LS] * alLS[idx_LS] * df(w0LS + waLS * alLS[idx_LS] - weLS_sq_div2)
        - (1 - qLS[idx_LS]) * alLS[idx_LS] * df(-w0LS - waLS * alLS[idx_LS] - weLS_sq_div2)
    )

    # grad[:, 14] corresponds to waLF
    grad[:, 14] = 0
    grad[idx_LF, 14] += (
        qLF[idx_LF] * alLF[idx_LF] * df(w0LF + waLF * alLF[idx_LF] - weLF_sq_div2)
        - (1 - qLF[idx_LF]) * alLF[idx_LF] * df(-w0LF - waLF * alLF[idx_LF] - weLF_sq_div2)
    )

    # Ensure no NaNs in gradient
    grad = np.nan_to_num(grad)

    return grad

In [8]:
# -----------------------------
# Methods/pruning.py
# -----------------------------

# Define the pruning function
def pruning(BD, LS, LF, sigma, side):
    """
    Pruning function to classify the local model based on BD, LS, and LF probabilities.

    Parameters:
    BD    : np.ndarray
        Binary indicator for BD.
    LS    : np.ndarray
        Landslide probabilities.
    LF    : np.ndarray
        Liquefaction probabilities.
    sigma : float
        Threshold parameter.
    side  : str
        Type of pruning ('single' or 'double').

    Returns:
    np.ndarray
        Local model classification.
    """

    model = np.zeros(BD.shape, dtype=int)

    if side == "single":
        # 1 - LS alone
        model += ((BD == 0) & (LF <= sigma)).astype(int)

        # 2 - LF alone
        model += 2 * ((BD == 0) & (LF > sigma) & (LF > LS)).astype(int)

        # 3 - LS and BD
        model += 3 * ((BD == 1) & (LF <= sigma)).astype(int)

        # 4 - LF and BD
        model += 4 * ((BD == 1) & (LF > sigma) & (LF > LS)).astype(int)
    else:
        # 1 - LS alone
        model += ((BD == 0) & (LS > LF + sigma) & (LS > 0)).astype(int)

        # 2 - LF alone
        model += 2 * ((BD == 0) & (LF > LS + sigma) & (LF > 0)).astype(int)

        # 3 - LS and BD
        model += 3 * ((BD == 1) & (LS > LF + sigma) & (LS > 0)).astype(int)

        # 4 - LF and BD
        model += 4 * ((BD == 1) & (LF > LS + sigma) & (LF > 0)).astype(int)

        # 5 - LF and LS
        model += 5 * ((BD == 0) & (np.abs(LF - LS) <= sigma)).astype(int)

        # 6 - LF and LS and BD
        model += 6 * ((BD == 1) & (np.abs(LF - LS) <= sigma)).astype(int)

    return model


In [9]:
# -----------------------------
# Methods/svi.py
# -----------------------------

# Define the SVI function
def SVI(Y, LS, LF, w, Nq, rho, delta, eps_0, LOCAL, lambda_, regu_type, sigma, prune_type):
    """
    Stochastic Variational Inference (SVI) function.

    Parameters:
    Y         : np.ndarray
        Observed data.
    LS        : np.ndarray
        Landslide probabilities.
    LF        : np.ndarray
        Liquefaction probabilities.
    w         : np.ndarray
        Initial weight vector.
    Nq        : int
        Number of Posterior Probability Iterations.
    rho       : float
        Step size for weight updates.
    delta     : float
        Acceptable tolerance for weight optimization.
    eps_0     : float
        Lower-bound non-negative weight.
    LOCAL     : np.ndarray
        Local model classification.
    lambda_   : float
        Regularization parameter.
    regu_type : int
        Type of regularization.
    sigma     : float
        Threshold for pruning.
    prune_type: str
        Type of pruning ('single' or 'double').

    Returns:
    tuple
        Optimized weights and posterior probabilities along with other outputs.
    """
    # Initialize variables
    QBD = 0.001 * np.random.rand(*Y.shape)
    QLS = LS.copy()
    QLF = LF.copy()
    loss = 1e+5
    loss_old = 0
    best_loss = -1e+4
    final_loss = []
    grad = np.zeros(len(w))
    epoch = 0

    # Compute the alpha that relates to prior LS and LF estimates
    t_alLS = np.log(LS / (1 - LS))
    t_alLF = np.log(LF / (1 - LF))
    t_alLS = np.clip(t_alLS, -6, 6)
    t_alLF = np.clip(t_alLF, -6, 6)
    final_w = w.copy()

    # Set stopping condition
    if np.sum(LOCAL >= 5) == 0:
        def myConditionFunc(x, y): return y > 0
    else:
        def myConditionFunc(x, y): return (x > 0) or (y > 0)

    # Main loop
    while (epoch < 30) and myConditionFunc(np.sum(LOCAL == 5), abs(loss_old - loss) - eps_0):
        # Create a sample batch
        totalnum = Y.size
        bsize = 500
        bnum = totalnum // bsize
        iD_sample = np.random.permutation(totalnum)
        iD = iD_sample[:bnum * bsize].reshape(bnum, bsize)

        # Reset loss parameters
        loss_old = loss
        loss = 0
        eloss = 0
        tmp_final_loss = []

        # Learning rate decay
        if (epoch % 10 == 0) and (epoch > 1):
            rho = max(rho * 0.1, 1e-4)

        # Iterate over batches
        for i in range(bnum):
            idx = np.unravel_index(iD[i], Y.shape)
            y = Y[idx]
            qBD = QBD[idx]
            qLS = QLS[idx]
            qLF = QLF[idx]
            local = LOCAL[idx]
            alLS = t_alLS[idx]
            alLF = t_alLF[idx]

            # Iterations
            for nq in range(Nq):
                # Apply the nonlinear T function
                q = 1 / (1 + np.exp(-Tfxn(y, qBD, qLS, qLF, alLS, alLF, w, local, delta)))
                qBD = q[:, 0]
                qLS = q[:, 1]
                qLF = q[:, 2]

                # Apply pruning
                qBD[local < 3] = 0
                qBD[local == 5] = 0
                qLS[np.isin(local, [0, 2, 4])] = 0
                qLF[np.isin(local, [0, 1, 3])] = 0

                # Remove negligible estimates
                qLS[qLS < 1e-6] = 0
                qLF[qLF < 1e-6] = 0

                # Apply sigma
                tqLS = qLS.copy()
                tqLF = qLF.copy()
                if prune_type == "single":
                    qLS[tqLS < tqLF - sigma] = 0
                    qLF[tqLF < tqLS] = 0
                else:
                    qLS[tqLS < tqLF - sigma] = 0
                    qLF[tqLF < tqLS - sigma] = 0

                # Update posterior estimates
                QBD[idx] = qBD
                QLS[idx] = qLS
                QLF[idx] = qLF

            # Classify new local model by pruning
            if np.sum(local >= 5) > 0:
                if prune_type == "single":
                    local[(tqLF < tqLS) & (local == 5)] = 1
                    local[(tqLF < tqLS) & (local == 6)] = 3
                    local[(tqLS < tqLF - sigma) & (local == 5)] = 2
                    local[(tqLS < tqLF - sigma) & (local == 6)] = 4
                else:
                    local[(tqLF < tqLS - sigma) & (local == 5)] = 1
                    local[(tqLF < tqLS - sigma) & (local == 6)] = 3
                    local[(tqLS < tqLF - sigma) & (local == 5)] = 2
                    local[(tqLS < tqLF - sigma) & (local == 6)] = 4
                LOCAL[idx] = local

            # Compute the partial derivative
            identity = np.column_stack([
                local >= 0, local >= 0, np.isin(local, [3, 4, 5]), np.isin(local, [1, 3, 5, 6]),
                np.isin(local, [2, 4, 5, 6]), np.isin(local, [3, 6]), np.isin(local, [4, 6]),
                np.isin(local, [3, 4, 6]), np.isin(local, [1, 3, 5, 6]), np.isin(local, [2, 4, 5, 6]),
                np.isin(local, [1, 3, 5, 6]), np.isin(local, [2, 4, 5, 6]), np.isin(local, [3, 4, 6]),
                np.isin(local, [1, 3, 5, 6]), np.isin(local, [2, 4, 5, 6])
            ])
            grad_D = parder(y, qBD, qLS, qLF, alLS, alLF, w, local)
            tmp_count = np.sum(identity, axis=0)
            grad = (np.sum(identity * grad_D, axis=0) / tmp_count)
            grad[tmp_count == 0] = 0

            # Compute the regularization
            if regu_type == 1:
                regu_grad = lambda_ * 100 * (1 / (1 + np.exp(-0.01 * w)) - 1 / (1 + np.exp(0.01 * w)))
                grad[[7, 8, 9]] -= regu_grad[[7, 8, 9]]
            else:
                regu_grad = lambda_ * 100 * (1 / (1 + np.exp(-0.01 * w)) - 1 / (1 + np.exp(0.01 * w)))
                grad[[13, 14]] -= regu_grad[[13, 14]]

            # Compute the new weight
            wnext = w + rho * grad
            wnext[[0, 2]] = np.minimum(wnext[[0, 2]], (-1e-6) * np.ones(2))
            wnext[[13, 14]] = np.maximum(0, wnext[[13, 14]])
            wnext[1] = np.clip(wnext[1], 1e-3, 1)

            # Compute the loss
            tmp_loss = np.mean(Loss(y, qBD, qLS, qLF, alLS, alLF, wnext, local, delta))
            loss += tmp_loss
            if i % 20 == 0:
                tmp_final_loss.append(tmp_loss)
            if i % 100 == 0:
                c_loss = np.mean(Loss(Y.flatten(), QBD.flatten(), QLS.flatten(), QLF.flatten(), t_alLS.flatten(), t_alLF.flatten(), wnext, LOCAL.flatten(), delta))
                if c_loss > best_loss:
                    final_QLS = QLS.copy()
                    final_QLF = QLF.copy()
                    final_QBD = QBD.copy()
                    final_w = wnext.copy()
                    best_loss = c_loss

            # Assign the new weight
            w = wnext.copy()

        # Show progress
        print(f"Epoch {epoch + 1}, loss: {loss / bnum}")
        epoch += 1
        final_loss.extend(tmp_final_loss)

    return final_w, final_QBD, final_QLS, final_QLF, QLS, QLF, QBD, final_loss, best_loss, LOCAL


# Evaluation

In [10]:
# -----------------------------
# Evaluation/binaryerror.py
# -----------------------------

# Define the binary error function
def binaryerror(prio, post, gtruth, prio_thresh, post_thresh):
    """
    Compute binary classification error metrics for prior and posterior.

    Parameters:
    prio       : np.ndarray
        Prior probabilities.
    post       : np.ndarray
        Posterior probabilities.
    gtruth     : np.ndarray
        Ground truth labels (0 or 1).
    prio_thresh: float
        Threshold for prior classification.
    post_thresh: float
        Threshold for posterior classification.

    Returns:
    tuple
        Metrics for prior and posterior:
        (prio_TPR, prio_FPR, prio_TNR, prio_FNR, prio_PRE,
         post_TPR, post_FPR, post_TNR, post_FNR, post_PRE)
    """

    # Compute TP, TN, FN, FP for prior
    prio_TP = np.sum((gtruth == 1) & (prio > prio_thresh))
    prio_TN = np.sum((gtruth == 0) & (prio <= prio_thresh))
    prio_FN = np.sum((gtruth == 1) & (prio <= prio_thresh))
    prio_FP = np.sum((gtruth == 0) & (prio > prio_thresh))

    # Compute TP, TN, FN, FP for posterior
    post_TP = np.sum((gtruth == 1) & (post > post_thresh))
    post_TN = np.sum((gtruth == 0) & (post <= post_thresh))
    post_FN = np.sum((gtruth == 1) & (post <= post_thresh))
    post_FP = np.sum((gtruth == 0) & (post > post_thresh))

    # Compute metrics for prior
    prio_TPR = prio_TP / (prio_TP + prio_FN) if (prio_TP + prio_FN) > 0 else 0
    prio_FPR = prio_FP / (prio_FP + prio_TN) if (prio_FP + prio_TN) > 0 else 0
    prio_TNR = prio_TN / (prio_TN + prio_FP) if (prio_TN + prio_FP) > 0 else 0
    prio_FNR = prio_FN / (prio_FN + prio_TP) if (prio_FN + prio_TP) > 0 else 0
    prio_PRE = prio_TP / (prio_TP + prio_FP) if (prio_TP + prio_FP) > 0 else 0

    # Compute metrics for posterior
    post_TPR = post_TP / (post_TP + post_FN) if (post_TP + post_FN) > 0 else 0
    post_FPR = post_FP / (post_FP + post_TN) if (post_FP + post_TN) > 0 else 0
    post_TNR = post_TN / (post_TN + post_FP) if (post_TN + post_FP) > 0 else 0
    post_FNR = post_FN / (post_FN + post_TP) if (post_FN + post_TP) > 0 else 0
    post_PRE = post_TP / (post_TP + post_FP) if (post_TP + post_FP) > 0 else 0

    return prio_TPR, prio_FPR, prio_TNR, prio_FNR, prio_PRE, post_TPR, post_FPR, post_TNR, post_FNR, post_PRE



In [11]:
# -----------------------------
# Evaluation/cel.py
# -----------------------------

# Define the cross-entropy loss function
def cel(prio, post, gtruth):
    """
    Compute the cross-entropy loss for prior and posterior.

    Parameters:
    prio   : np.ndarray
        Prior probabilities.
    post   : np.ndarray
        Posterior probabilities.
    gtruth : np.ndarray
        Ground truth labels (0 or 1).

    Returns:
    tuple
        Cross-entropy loss for prior and posterior (ploss, qloss).
    """
    tmp_prio = (prio - np.min(prio)) / (np.max(prio) - np.min(prio))
    tmp_prio = np.clip(tmp_prio, 1e-6, 1 - 1e-6)
    ploss = -np.mean(gtruth * np.log(tmp_prio) + (1 - gtruth) * np.log(1 - tmp_prio))

    tmp_post = (post - np.min(post)) / (np.max(post) - np.min(post))
    tmp_post = np.clip(tmp_post, 1e-6, 1 - 1e-6)
    qloss = -np.mean(gtruth * np.log(tmp_post) + (1 - gtruth) * np.log(1 - tmp_post))

    return ploss, qloss

In [12]:
# -----------------------------
# Evaluation/rocdetpr.py
# -----------------------------

# Define the ROC, DET, PR curve function
def rocdetpr(title, prio, post, gtruth, location):
    """
    Plot ROC, DET, and PR curves for prior and posterior probabilities.

    Parameters:
    title     : str
        Title for the plots.
    prio      : np.ndarray
        Prior probabilities.
    post      : np.ndarray
        Posterior probabilities.
    gtruth    : np.ndarray
        Ground truth labels (0 or 1).
    location  : str
        Directory to save the plots.

    Returns:
    np.ndarray
        Array containing metrics for various thresholds.
    """
    acc = []
    ith_percentile = np.arange(0, 1.01, 0.01)

    for l in ith_percentile:
        # Compute thresholds
        prio_thresh = np.quantile(prio.flatten(), l)
        post_thresh = np.quantile(post.flatten(), l)

        # Compute error metrics
        prio_TPR, prio_FPR, prio_TNR, prio_FNR, prio_PRE, post_TPR, post_FPR, post_TNR, post_FNR, post_PRE = binaryerror(
            prio, post, gtruth, prio_thresh, post_thresh)

        # Count the number of estimates above the threshold
        tmp_p_c = np.sum(prio > prio_thresh)
        tmp_q_c = np.sum(post > post_thresh)

        # Append
        tmp_acc = [tmp_p_c, tmp_q_c, prio_TPR, prio_FPR, prio_TNR, prio_FNR, prio_PRE,
                   post_TPR, post_FPR, post_TNR, post_FNR, post_PRE]
        acc.append(tmp_acc)

    acc = np.array(acc)

# Main

In [ ]:
# -----------------------------
# Main/main_run.py
# -----------------------------

# Define Main script
def main():
    # Initialize
    # Import data
    location = '/content/'
    with rasterio.open(f'{location}PR_BD.tif') as src:
        Y = src.read(1)
        Y_R = src.transform

    with rasterio.open(f'{location}PR_BD.tif') as src:
        BD = src.read(1)
        BD_R = src.transform

    with rasterio.open(f'{location}PR_LS.tif') as src:
        LS = src.read(1)
        LS_R = src.transform

    with rasterio.open(f'{location}PR_LF.tif') as src:
        LF = src.read(1)
        LF_R = src.transform

    # Fix Data Input
    BD[BD > 0] = 1
    Y[np.isnan(Y)] = 0
    BD[np.isnan(BD)] = 0
    LS[np.isnan(LS)] = 0
    LF[np.isnan(LF)] = 0

    # Convert Landslide Areal Percentages to Probabilities
    new_LS = LS.copy()
    index = np.where(LS > 0)
    for i in zip(*index):
        p = [4.035, -3.042, 5.237, (-7.592 - np.log(LS[i]))]
        roots = np.roots(p)
        new_LS[i] = np.real(roots[np.isreal(roots)][0])

    # Convert Liquefaction Areal Percentages to Probabilities
    new_LF = LF.copy()
    index = np.where(LF > 0)
    for i in zip(*index):
        new_LF[i] = (np.log((np.sqrt(0.4915 / LF[i]) - 1) / 42.40)) / (-9.165)

    # Change into Non-negative Probabilities
    new_LF[new_LF < 0] = 0
    new_LS[new_LS < 0] = 0
    new_LS[np.isnan(new_LS)] = 0
    new_LF[np.isnan(new_LF)] = 0
    tmp_LF = new_LF.copy()
    tmp_LS = new_LS.copy()

    # Classify Local Model by Pruning
    prune_type = 'double'
    sigma = np.median(np.abs(new_LS[(LS > 0) & (LF > 0)] - new_LF[(LS > 0) & (LF > 0)]))
    LOCAL = pruning(BD, tmp_LS, tmp_LF, sigma, prune_type)
    tmp_LS[(LOCAL == 5) | (LOCAL == 6)] = np.min(new_LS[new_LS > 0])
    tmp_LF[(LOCAL == 5) | (LOCAL == 6)] = np.min(new_LF[new_LF > 0])

    # Set Lambda Term
    lambda_ = 0

    # Initialize Weight Vector w
    w = np.random.rand(15)
    w[[3, 4]] = 0
    w[[0, 2]] = -1 * w[[0, 2]]
    regu_type = 1

    # Set Variational hyperparameters
    Nq = 10  # Number of Posterior Probability Iterations

    # Set Weight Updating Parameters
    rho = 1e-3  # Step size
    delta = 1e-3  # Acceptable tolerance for weight optimization
    eps_0 = 0.001  # Lower-bound non-negative weight

    # Output
    opt_w, opt_QBD, opt_QLS, opt_QLF, QLS, QLF, QBD, final_loss, best_loss, local = SVI(
        Y, tmp_LS, tmp_LF, w, Nq, rho, delta, eps_0, LOCAL, lambda_, regu_type, sigma, prune_type)

    # Convert Probabilities to Areal Percentages
    final_QLS = np.exp(-7.592 + 5.237 * opt_QLS - 3.042 * opt_QLS ** 2 + 4.035 * opt_QLS ** 3)
    final_QLF = 0.4915 / (1 + 42.40 * np.exp(-9.165 * opt_QLF)) ** 2

    # Round down very small areal percentages to zero
    final_QLS[final_QLS <= np.exp(-7.592)] = 0
    final_QLF[final_QLF <= 0.4915 / (1 + 42.40) ** 2] = 0

    # Remove probabilities in water bodies
    final_QLS[(LS == 0) & (LF == 0)] = 0
    final_QLF[(LS == 0) & (LF == 0)] = 0

    # Export GeoTIFF Files
    with rasterio.open('QLS.tif', 'w', driver='GTiff', height=final_QLS.shape[0],
                       width=final_QLS.shape[1], count=1, dtype=final_QLS.dtype,
                       crs=LS_R.crs, transform=LS_R.transform) as dst:
        dst.write(final_QLS, 1)

    with rasterio.open('QLF.tif', 'w', driver='GTiff', height=final_QLF.shape[0],
                       width=final_QLF.shape[1], count=1, dtype=final_QLF.dtype,
                       crs=LS_R.crs, transform=LS_R.transform) as dst:
        dst.write(final_QLF, 1)

    with rasterio.open('QBD.tif', 'w', driver='GTiff', height=QBD.shape[0],
                       width=QBD.shape[1], count=1, dtype=QBD.dtype,
                       crs=LS_R.crs, transform=LS_R.transform) as dst:
        dst.write(QBD, 1)

    # Save all to a file
    np.savez(f'{location}lambda{lambda_}_sigma{sigma}_prune{prune_type}.npz', opt_w=opt_w,
             opt_QBD=opt_QBD, opt_QLS=opt_QLS, opt_QLF=opt_QLF, QLS=QLS, QLF=QLF, QBD=QBD,
             final_loss=final_loss, best_loss=best_loss, LOCAL=LOCAL)

if __name__ == "__main__":
    main()